In [4]:
suppressPackageStartupMessages({
    library(sf)
    # library(spatula)
    library(ggthemes)
    library(purrr)
    library(glue)
    library(furrr)
    library(sfarrow)
    library(future)
    library(dplyr)
    library(data.table)
    library(ggplot2)
    library(igraph)
    library(Rcpp)
})


fig.size <- function(h, w) {
    options(repr.plot.height = h, repr.plot.width = w)
}

In [52]:
# devtools::load_all('..')

In [2]:
devtools::document('..')

ℹ Updating spatula documentation

ℹ Loading spatula

Warning message:
“infer_polygons.Rd is missing name/title. Skipping”


Writing NAMESPACE


In [1]:
## git add . --all
## git commit -m '...'
devtools::install_git('..')

ℹ Updating spatula documentation

ℹ Loading spatula

Loading required package: Rcpp

Loading required package: RcppArmadillo



# Data

In [43]:
load('../data/spatula_cosmx.rda')

In [44]:
head(tx)
nrow(tx)

gene,cell,x,y
<chr>,<int>,<dbl>,<dbl>
KRT86,0,357.8340,232.1760
KRT86,2642,499.2180,222.3600
KRT86,1024,498.4053,315.8624
KRT86,0,379.0131,365.4386
KRT86,1097,446.7708,294.9840
KRT86,1343,424.6807,224.5995


[1] 153214

In [45]:
data.frame(head(cells))
nrow(cells)

,id,shape,neighborhood,ntx_cell,ngene_cell,ntx_neighbors,ngene_neighbors
,<chr>,<POLYGON>,<POLYGON>,<dbl>,<int>,<dbl>,<int>
1,920,POLYGON ((492.0364 347.7475...,POLYGON ((469.1623 366.1447...,539,198,6814,726
2,923,POLYGON ((362.5765 349.5369...,POLYGON ((339.2242 352.7786...,236,128,4567,682
3,935,POLYGON ((409.3373 349.9752...,"POLYGON ((384.419 356.0701,...",476,187,6660,759
4,946,"POLYGON ((520.715 338.3761,...",POLYGON ((500.3057 334.5539...,516,183,7683,737
5,953,POLYGON ((497.6366 348.2479...,POLYGON ((473.1845 337.8278...,233,114,7025,720
6,956,"POLYGON ((414.061 334.1596,...","POLYGON ((406.544 315.5595,...",800,283,7990,771


[1] 202

# Run it 

In [46]:
output_dir='tmp'
fname_baysor_config='config.toml'
# baysor_binary='/n/data1/bwh/medicine/korsunsky/lab/rom4535/software/baysor'
opts = list(
    baysor_mode = 'docker', ## baysor can be run through docker container or natively installed binary 
    max_tx_per_grid = 10000, ## maximum number of transcripts per tile 
    compute_shapes = TRUE, 
    max_parallel_jobs = 60, ## maximum number of concurrent Baysor jobs running 
    min_tx_per_cell = 10, 
    remove_temp_files = TRUE ## set to FALSE for debugging 
)

# baysor.run_baysor_tiles_binary <- function(ntiles, output_dir, fname_baysor_config, max_parallel_jobs) {

# }


In [47]:
ntiles <- baysor.split_tx_files(output_dir, opts$max_tx_per_grid) 
baysor.prepare_cmds(ntiles, output_dir, fname_baysor_config, opts$max_parallel_jobs)
baysor.run_baysor_tiles_docker(ntiles, output_dir, fname_baysor_config, max_parallel_jobs=opts$max_parallel_jobs) 


In [51]:
tx <- baysor.collect_tx(output_dir, testing=FALSE, mintx=opts$min_tx_per_cell) 
counts <- tx_to_counts(tx$gene, tx$cell, remove_bg = TRUE)
cells <- baysor.collect_cells(tx, opts$min_tx_per_cell)
environment(baysor.finish) <- environment() 
baysor.finish() 

Warning message:
“This is an initial implementation of Parquet/Feather file support and
geo metadata. This is tracking version 0.1.0 of the metadata
(https://github.com/geopandas/geo-arrow-spec). This metadata
specification may change and does not yet make stability promises.  We
do not yet recommend using this in a production setting unless you are
able to rewrite your Parquet/Feather files.”


In [67]:
list.files('tmp')

st_read_parquet('tmp/shapes.parquet') %>% head(5) %>% data.frame()
fread('tmp/cells.csv', nrows = 5)
spatula::readMM('tmp/counts.mtx')[1:5, 1:5]
readLines('tmp/genes.tsv', n = 5)
fread('tmp/transcripts.csv', nrows = 5)

[1] "cells.csv"       "counts.mtx"      "genes.tsv"       "shapes.parquet" 
[5] "transcripts.csv"

,id,shape
,<chr>,<POLYGON>
1,1,POLYGON ((351.9471 171.8564...
2,2,POLYGON ((363.3103 178.4181...
3,3,POLYGON ((353.7744 196.3305...
4,4,POLYGON ((370.3212 194.9418...
5,5,"POLYGON ((378.1629 171.154,..."


id,x,y,area,n_transcripts,avg_confidence,cluster,rand_color
<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<chr>
1,355.8171,178.7981,92.126717,672,0.8633929,3,#e377c2
2,366.9396,183.4787,101.011793,411,0.8758637,4,#ff9896
3,355.1113,197.5212,5.415452,24,0.8175000,2,#9edae5
4,363.0144,194.3561,155.102557,449,0.7912249,3,#9edae5
5,379.5477,178.3871,124.073594,793,0.8587390,3,#1f77b4


5 x 5 sparse Matrix of class "dgCMatrix"
              
[1,] . . . . .
[2,] . . . . .
[3,] . . . . .
[4,] . . . . .
[5,] . 1 . . .

[1] "AATK" "ABL1" "ABL2" "ACE"  "ACE2"

gene,cell,x,y,molecule_id,prior_segmentation,confidence,cluster,assignment_confidence,is_noise,ncv_color,N
<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<chr>,<int>
KRT86,8,381.2092,197.3803,1,2689,0.99996,1,0.92,FALSE,#00B7D3,205
KRT86,16,388.4400,219.5220,2,1366,1.00000,1,1.00,FALSE,#009BD9,226
KRT86,15,378.7273,216.1332,3,1361,0.99998,3,0.48,FALSE,#002F45,462
KRT86,16,389.6978,215.4735,4,1366,1.00000,1,0.94,FALSE,#0083AA,226
BECN1,7,386.7510,177.4740,5,0,1.00000,3,0.56,FALSE,#730E7E,600
